In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/final_ratings.csv
/kaggle/input/amazon-books-reviews/books_data.csv
/kaggle/input/amazon-books-reviews/Books_rating.csv


## Loading the Dataset

In [2]:
books = pd.read_csv('/kaggle/input/amazon-books-reviews/books_data.csv')
ratings = pd.read_csv('/kaggle/input/amazon-books-reviews/Books_rating.csv')

In [3]:
books.shape,ratings.shape

((212404, 10), (3000000, 10))

In [4]:
books.head(2)

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN


In [5]:
ratings.head(2)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...


## Collaborative Filtering

In [6]:
ratings = ratings[['Id','User_id','review/score','Title']]

In [7]:
ratings.rename(columns={'Id':'book_id','User_id':'user_id','review/score':'rating','Title':'title'},inplace=True)

In [8]:
ratings.head(2)

,book_id,user_id,rating,title
0,1882931173,AVCGYZL8FQQTD,4.0,Its Only Art If Its Well Hung!
1,0826414346,A30TK6U7DNS82R,5.0,Dr. Seuss: American Icon


### Calculating the number of ratings in of each book

In [9]:
num_rating_df = ratings.groupby('title').count()['rating'].reset_index()
num_rating_df.rename(columns={'rating':'num_rating'},inplace=True)
num_rating_df.head()

,title,num_rating
0,""" Film technique, "" and, "" Film acting """,2
1,""" We'll Always Have Paris"": The Definitive Gui...",2
2,"""... And Poetry is Born ..."" Russian Classical...",1
3,"""A Titanic hero"" Thomas Andrews, shipbuilder",8
4,"""A Truthful Impression of the Country"": Britis...",1


In [10]:
# avg_rating_df = ratings.groupby('title').mean()['rating'].reset_index()
# avg_rating_df.rename(columns={'rating':'avg_rating'},inplace=True)
# avg_rating_df

#### Filtering the data on the basis of users and books

In [11]:
x = ratings.groupby('user_id').count()['rating'] > 200
considerable_users = x[x].index

In [12]:
filtered_rating = ratings[ratings['user_id'].isin(considerable_users)]

In [13]:
filtered_rating

,book_id,user_id,rating,title
1,0826414346,A30TK6U7DNS82R,5.0,Dr. Seuss: American Icon
6,0826414346,A14OJS0VWMOSWO,5.0,Dr. Seuss: American Icon
59,0963923080,A3NIQK6ZLYEP1L,4.0,Rising Sons and Daughters: Life Among Japan's ...
140,0789480662,A281NPSIMI1C2R,5.0,Eyewitness Travel Guide to Europe
176,1884734766,A14OJS0VWMOSWO,5.0,Dealing With Disappointment: Helping Kids Cope...
...,...,...,...,...
2999935,0786182431,A2CR57GAJKNWVV,4.0,Very Bad Deaths: Library Edition
2999937,0786182431,A1I2O9Y3X3HXLS,5.0,Very Bad Deaths: Library Edition
2999949,0786182431,AFVQZQ8PW0L,5.0,Very Bad Deaths: Library Edition
2999960,0679417052,A9Q28YTLYREO7,5.0,My Life


In [14]:
y = filtered_rating.groupby('title').count()['rating']>=50
famous_books = y[y].index

In [15]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_books)]

In [16]:
final_ratings

,book_id,user_id,rating,title
1747,B000N6DDJQ,AUM3YMZ0YRJE0,5.0,The Scarlet Letter A Romance
1807,B000N6DDJQ,AMKZHBOK7VMQR,5.0,The Scarlet Letter A Romance
1820,B000N6DDJQ,AWLFVCT9128JV,4.0,The Scarlet Letter A Romance
1829,B000N6DDJQ,ABN5K7K1TM1QA,5.0,The Scarlet Letter A Romance
1830,B000N6DDJQ,A3IKBHODOTYYHM,4.0,The Scarlet Letter A Romance
...,...,...,...,...
2999633,B000P91JYW,A1D2C0WDCSHUWZ,5.0,Jane Eyre
2999638,B000P91JYW,AHUT55E980RDR,4.0,Jane Eyre
2999641,B000P91JYW,A1T17LMQABMBN5,5.0,Jane Eyre
2999646,B000P91JYW,AJQ1S39GZBKUG,5.0,Jane Eyre


In [17]:
# saving the final dataset
final_ratings.to_csv('final_ratings.csv',index=False)

## Loading the final saved Dataset

In [18]:
import pandas as pd
import numpy as np

In [19]:
final_df = pd.read_csv('/kaggle/input/book-recommendation-dataset/final_ratings.csv')

In [20]:
final_df.head()

,book_id,user_id,rating,title
0,B000N6DDJQ,AUM3YMZ0YRJE0,5.0,The Scarlet Letter A Romance
1,B000N6DDJQ,AMKZHBOK7VMQR,5.0,The Scarlet Letter A Romance
2,B000N6DDJQ,AWLFVCT9128JV,4.0,The Scarlet Letter A Romance
3,B000N6DDJQ,ABN5K7K1TM1QA,5.0,The Scarlet Letter A Romance
4,B000N6DDJQ,A3IKBHODOTYYHM,4.0,The Scarlet Letter A Romance


### Creating the Pivot Table

In [21]:
pt = final_df.pivot_table(index='user_id',columns='title',values='rating')

In [22]:
pt

title,1984,A Christmas Carol (Classic Fiction),A Connecticut Yankee in King Arthur's Court,A Farewell to Arms,A Fine Balance,A Journey to the Center of the Earth,A Portrait of the Artist As a Young Man,A Portrait of the Artist as a Young Man,A Princess of Mars,A Tree Grows in Brooklyn,...,"Ulysses,",Ulysses.,Wuthering Heights,Wuthering Heights (College classics in English),Wuthering Heights (New Windmill),Wuthering Heights (Penguin Audiobooks),Wuthering Heights (Riverside editions),Wuthering Heights (Signet classics),Wuthering Heights (The Franklin Library),Wuthering Heights.
user_id,,,,,,,,,,,,,,,,,,,,,
A106016KSI0YQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10T0OW97SFBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
A114YQ7ZT9Y1W5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A11DCTGTPS7M0C,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A126KX6FVI4T66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AV74NYPSKHXBU,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
AWLFVCT9128JV,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AX54G5AL870Q8,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
pt.fillna(0,inplace=True)

In [24]:
pt

title,1984,A Christmas Carol (Classic Fiction),A Connecticut Yankee in King Arthur's Court,A Farewell to Arms,A Fine Balance,A Journey to the Center of the Earth,A Portrait of the Artist As a Young Man,A Portrait of the Artist as a Young Man,A Princess of Mars,A Tree Grows in Brooklyn,...,"Ulysses,",Ulysses.,Wuthering Heights,Wuthering Heights (College classics in English),Wuthering Heights (New Windmill),Wuthering Heights (Penguin Audiobooks),Wuthering Heights (Riverside editions),Wuthering Heights (Signet classics),Wuthering Heights (The Franklin Library),Wuthering Heights.
user_id,,,,,,,,,,,,,,,,,,,,,
A106016KSI0YQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10T0OW97SFBB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
A114YQ7ZT9Y1W5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11DCTGTPS7M0C,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A126KX6FVI4T66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AV74NYPSKHXBU,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
AWLFVCT9128JV,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AX54G5AL870Q8,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Finding the Simlarities among the users

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
similarity_scores = cosine_similarity(pt)

In [27]:
similarity_scores


array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.07151985, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07151985, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.22086305,
        0.18586518],
       [0.        , 0.        , 0.        , ..., 0.22086305, 1.        ,
        0.42077019],
       [0.        , 0.        , 0.        , ..., 0.18586518, 0.42077019,
        1.        ]])

In [28]:
similarity_scores.shape

(298, 298)

In [29]:
np.where(pt.index=='A106016KSI0YQ')[0][0]

0

In [30]:
sorted(list(enumerate(similarity_scores[0])),key=lambda x:x[1],reverse=True)[1:6]

[(290, 1.0),
 (57, 0.37796447300922725),
 (5, 0.25683274846875703),
 (138, 0.24514516892273006),
 (212, 0.23211917272131485)]

In [31]:
pt.index[290]

'AUHG8KSHI529U'

In [32]:
df = ratings[ratings['user_id'] == pt.index[290]].sort_values(by='rating',ascending=False)
# lt = list(df[df["review/score"].isin([4.0,5.0])].head()['Title'])
# lt
df.shape

(298, 4)

In [33]:
user_books_lst = list(ratings[ratings['user_id'] == pt.index[0]]['title'])
df = df[~df['title'].isin(user_books_lst)]
df = df[df['rating'].isin([4.0,5.0])]
lst = list(df.head()['title'])

In [34]:
data = []
data.extend(lst)

In [35]:
data.extend(['Jungle Book'])
data

['Consilience (University Press Audiobooks)',
 'The New Americans: How the Melting Pot Can Work Again',
 'Beethoven (Oxford paperbacks)',
 'Counterpoint',
 "Shakespeare's Macbeth; (Macmillan's English classics)",
 'Jungle Book']

### Function to recommend books to a user

In [36]:
def recommend(user_id):
    # index fetch
    index = np.where(pt.index==user_id)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:4]
    
    data = []
    for i in similar_items:
        df = ratings[ratings['user_id'] == pt.index[i[0]]].sort_values(by='rating',ascending=False)
        user_books_lst = list(ratings[ratings['user_id'] == pt.index[index]]['title'])
        df = df[~df['title'].isin(user_books_lst)]
        df = df[df['rating'].isin([4.0,5.0])]
        data.extend(list(df.head()['title']))
        data = [*set(data)]
    return data

In [37]:
recommend('A106016KSI0YQ')

['A striving after wind',
 'House of the Sleeping Beauties',
 'A Case Of Conscience',
 'The Scarlet Letter A Romance',
 'The Rebel',
 'Sea Wolf',
 'All quiet on the western front;',
 "Shakespeare's Macbeth; (Macmillan's English classics)",
 'Consilience (University Press Audiobooks)',
 'Beethoven (Oxford paperbacks)',
 'Counterpoint',
 'Taiji Chin Na: The Seizing Art of Taijiquan (Chinese Internal Martial Arts)',
 'The New Americans: How the Melting Pot Can Work Again',
 'Small-Circle Jujitsu',
 'SPACE, TIME AND ARCHITECTURE: THE GROWTH OF A NEW TRADITION.']

In [38]:
recommend('AX54G5AL870Q8')

["The Fort at River's Bend: The Sorcerer, Book 1 (The Camulod Chronicles, Book 5)",
 'Snuggle Puppy!',
 "Tomorrow's Alphabet",
 'The Attack',
 'The Dragon Machine',
 'In a Dark House (Crombie, Deborah)',
 'Tough Boris',
 'The Nature of Water and Air',
 'Pajama Time!',
 'THE DOLLMAKER',
 'Cotillion',
 'Winter Solstice',
 "Myron's Magic Cow",
 'Long Spoon Lane (Charlotte & Thomas Pitt Novels)',
 "All the King's Men"]